In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
path_to_wheat_data = "/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/wheat_final_data.csv"

wheat_data = pd.read_csv(path_to_wheat_data)

In [ ]:
wheat_data

In [ ]:
# name of querytype for which data is required to be extracted
query_type_to_work_upon = "seed treatment"

data = wheat_data[ (wheat_data['QueryType']== query_type_to_work_upon) ]

# path to the folder in drive for queryType
path = "/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/seed treatment/"

# path to save queryType data
path_saving_query_type = path + query_type_to_work_upon + ".csv"
data.to_csv(path_saving_query_type, index=False)

In [ ]:
data = pd.read_csv(path_saving_query_type)

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 3.5 MB 9.1 MB/s 
     |████████████████████████████████| 6.8 MB 35.0 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=5cce2f589affe37d7a5a4ca23a87c48c2e10062eceadba85d19725b80149cbca
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

model = SentenceTransformer('all-MiniLM-L6-v2')

def get_unique_query_text(df, model):
  print(df.head)
  test_list = [df["QueryText"][0]]
  print(test_list)
  for m,ques1 in enumerate(df["QueryText"][1:]):
      emb1 = model.encode(ques1)
      temp_list = []
      for element in test_list:
          emb2 = model.encode(element)
          cos_sim = util.cos_sim(emb1, emb2)
          if cos_sim.item() < 0.8:
              temp_list.append(ques1)
      if len(temp_list) == len(test_list):
          test_list.append(ques1)
  return test_list

In [ ]:
# function call to get unique data
test_list = get_unique_query_text(data, model)

# path to save the unique questions
path_to_save_unique_questions = path + 'wheat_{}_final_unique_ques.csv'.format(query_type_to_work_upon)

df_unique_ques = pd.DataFrame({'ques' : test_list})
df_unique_ques.to_csv(path_to_save_unique_questions , index = False)

<bound method NDFrame.head of          Sector Category  ...               CreatedOnd month
0   agriculture  cereals  ...  2012-10-10 14:32:00.000  10.0
1   agriculture  cereals  ...  2012-10-30 08:00:00.000  10.0
2   agriculture  cereals  ...  2011-11-09 11:47:00.000  11.0
3   agriculture  cereals  ...  2011-11-10 09:13:00.000  11.0
4   agriculture  cereals  ...  2012-11-07 06:57:00.000  11.0
..          ...      ...  ...                      ...   ...
57  agriculture  cereals  ...  2011-11-02 16:23:00.000  11.0
58  agriculture  cereals  ...  2012-11-08 06:33:00.000  11.0
59  agriculture  cereals  ...  2012-12-08 11:52:00.000  12.0
60  agriculture  cereals  ...  2012-10-27 12:04:00.000  10.0
61  agriculture  cereals  ...  2012-11-02 15:17:00.000  11.0

[62 rows x 11 columns]>
['information regarding to the seed treatment of wheat ?']


## After generating the unique data, we need to check the saved csv manually too as there still can be some unwanted sentences.

In [ ]:
unique = pd.read_csv(path_to_save_unique_questions)

## Parapharsing

Installing required modules

In [ ]:
!pip install torch==1.4.0
!pip install transformers==2.9.0
!pip install pytorch_lightning==0.7.5

Setting up the model and downloading weights. (Size is more than 1 GB)

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

device  cuda


In [ ]:
import pandas as pd
def parapharsing_data(df, model, tokenizer):
  dict_ques_ans = {}
  for query in df["ques"]:
      #print("The Query is", query)
      text = str(query) + " </s>"
      max_len = 64
      encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
      input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
      beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=64,
      top_k=50,
      top_p=0.95,
      early_stopping=True,
      num_return_sequences=10
      )
      final_outputs =[]
      for beam_output in beam_outputs:
          sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
          if sent.lower() != text.lower() and sent not in final_outputs:
              final_outputs.append(sent)

      dict_ques_ans[query] = final_outputs
  return dict_ques_ans

#loading the given csv file
# df = pd.read_csv(path_to_save_unique_questions)
df = pd.read_csv("/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/seed treatment/wheat_seed treatment_final_unique_ques.csv")
# function call
dict_ques_ans = parapharsing_data(df, model, tokenizer)

In [ ]:
#code to write paraphrases in required format.
import csv
final_text = ""
with open('paraphrased_final.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for ques in dict_ques_ans:
        final_text = "    - " + str(ques) + '\n' + '    - '
        csvwriter.writerow([final_text + '\n    - '.join(dict_ques_ans[ques])])

In [ ]:
# copying parapharased question file to drive
!cp "paraphrased_final.csv"  "/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/seed treatment/"

## after saving the parapharsed file one shoud check the quality of data generated and update as required for further processing.

In [ ]:
unique_quest_file_path = "/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/seed treatment/wheat_seed treatment_final_unique_ques.csv"
para_file = "/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/seed treatment/paraphrased_final.csv"

In [ ]:
import re

def generate_yml_files(intent_list, domain_list, nlu_file_name, domain_file_name, retrival_name):
  # nlu file
  with open(nlu_file_name, "w") as nlu:
    nlu.write('version: \"3.0\" \n')
    nlu.write("nlu: \n")
    nlu.writelines(intent_list)
  
  # domain file
  with open(domain_file_name, "w") as domain:
    domain.write('version: \"3.0\" \n')
    domain.write("intents: \n")
    domain.write("  - {}\n".format(retrival_name))
    domain.write("responses:\n")
    domain.writelines(domain_list)

def generate_data(retrival_name, uni_ques, paraData):
  intent_list = []
  domain_list = []
  for index, row in uni_ques.iterrows():
    qt = row["ques"]
    text = re.sub('[^a-zA-Z0-9 ]', '', qt.strip())
    text = text.strip()
    result = retrival_name + "/" + text.replace(" ", "-")
    intent = "\n- intent: " + result + "\n" + "  examples: |\n"
    examples = paraData[0].iloc[index]
    final = intent + examples
    # print(final)
    intent_list.append(final)
    # ------- Domain Data -----------------
    domain = "  utter_" + result + ": \n" + "  - text: \n"
    # print(domain)
    domain_list.append(domain)
  
  return intent_list, domain_list


In [ ]:
# parameters
retrival_name = "faq-agri-wheat-b4"
nlu_file_name = "nlu-faq-agri-wheat-b4.yml"
domain_file_name = "domain-agri-wheat-b4.yml"
uni_ques = pd.read_csv(unique_quest_file_path)
paraData = pd.read_csv(para_file, header=None)

# function call
# it will create two lists having nlu and domain file data
intent_list, domain_list = generate_data(retrival_name, uni_ques, paraData)
# this function will generate yml files for nlu and domain
generate_yml_files(intent_list, domain_list, nlu_file_name, domain_file_name, retrival_name)